# Recommender Systems 2022/2023

## Practice Session 11 - MF with PyTorch

PyTorch, Tensorflow, Keras are useful framework that allow you to build machine learning models (from linear regression to complex deep learning methods) and hide almost all of the complexity related to the training. Usually, you only have to create an object that starting from the model parameters will be able to compute your prediction, then specify the loss and the framework automatically calculates the gradients.

#### Performance warning!
In image processing tasks one usually has an image, maybe a reasonably large one (1000x1000x3, hence 3\*10^6 data points) on which a complex network is applied (multiple convolution operations, pooling etc). The computationally expensive part can be effectively parallelized by the framework and hence the speedup over using single-core operations is massive.

Unfortunately here we are not dealing with images, but with user profiles. In terms of data this means each profile can be in the 10^5 - 10^6 items. The issue arises when considering the model. If you use a matrix factorization model, the core of the operation is a dot product between two embedding vectors, which is an extremely fast operation. There is hardly any speedup and the burden of the overall infrastructure is not offset by it. For this reason, if you use a profiler you will see that 80-90% of the time is spent in the data sampling phase (because it is done in python it can be quite slow) and the actual prediction computation is a tiny fraction of the time. Overall, a *simple* matrix factorization model may be 10x slower if implemented with pytorch. The gap will reduce if you use a powerful GPU but I am yet to find someone able to run that on a GPU faster than the Cython single-core implementation.

#### Prototyping
Given how the complexity of gradients and such is hidden, pytorch becomes a great tool for prototyping. It is very easy to change someting in your model becayse you do not need to dig in Cython code. For example, you may implement a SLIM MSE method that uses as inital parameters the similarity computed with an item-based cosine method, or you may create a hybrid of multiple similarities and lear the weights to use for each similarity.

## What do we need

* A Dataset object to load the data
* Model object
* Training loop

In [2]:
import scipy.sparse as sps

from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.KNN import ItemKNNCustomSimilarityRecommender
from challenge.utils.functions import read_data, generate_submission_csv

data_file_path = '../challenge/input_files/data_train.csv'
users_file_path = '../challenge/input_files/data_target_users_test.csv'
URM_all_dataframe, users_list = read_data(data_file_path, users_file_path)

URM_all = sps.coo_matrix(
    (URM_all_dataframe['Data'].values, (URM_all_dataframe['UserID'].values, URM_all_dataframe['ItemID'].values)))
URM_all = URM_all.tocsr()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.80)

### MF models rely upon latent factors for users and items which are called 'embeddings'

![latent factors](https://miro.medium.com/max/988/1*tiF4e4Y-wVH732_6TbJVmQ.png)

In [3]:
num_factors = 10

n_users, n_items = URM_train.shape

In [4]:
import torch

# Creates U
user_factors = torch.nn.Embedding(num_embeddings=n_users, embedding_dim=num_factors)

# Creates V
item_factors = torch.nn.Embedding(num_embeddings=n_items, embedding_dim=num_factors)

In [5]:
user_factors

Embedding(13025, 10)

In [6]:
item_factors

Embedding(22348, 10)

## In order to compute the prediction you have to:
- Get a list of user and item indices (as tensors)
- Get the user and item embedding
- Compute the element-wise product of the embeddings

In [7]:
user_index = torch.Tensor([42]).type(torch.LongTensor)
item_index = torch.Tensor([42]).type(torch.LongTensor)

user_index, item_index

(tensor([42]), tensor([42]))

### Notice that each object has a "grad_fn=..." attribute, which si going to be used for the automatic gradient compuation to go backwards in the operations required to compute the prediction.

In [8]:
current_user_factors = user_factors(user_index)
current_item_factors = item_factors(item_index)

current_user_factors, current_item_factors

(tensor([[ 1.1120,  0.0877, -0.7391,  0.9084, -0.3965, -2.2140, -0.4592, -0.6728,
          -0.7141,  0.8734]], grad_fn=<EmbeddingBackward0>),
 tensor([[-0.8987,  0.0286,  0.0557, -0.3426,  1.5130, -0.9608,  0.9584,  0.5459,
          -0.0544,  1.0602]], grad_fn=<EmbeddingBackward0>))

Now the dot product is just a summation over the elementwise product. 

In [9]:
prediction = torch.mul(current_user_factors, current_item_factors).sum()
prediction

tensor(0.3355, grad_fn=<SumBackward0>)

Notice how the "grad_fn" states "SubBackward", the prediction was indeed due to a sum

#### We can also use the einstein summation format, which is particularly useful when you have a more complex equation to compute the prediction

The einstein summation allows you to write the prediction in terms of the indices of a summation. In this case we want to iterate both embedding vectors, perform an element-by-element product and then sum at the end. Be careful on the dimensions, in this case the factors have two dimensions (the row dimension is 1 so in practice it is useless). We use "b" to iterate over the rows (useful when we compute batches of predictions to parallelize) and "i" is the latent factor index.

In [10]:
torch.einsum("bi,bi->b", current_user_factors, current_item_factors)

tensor([0.3355], grad_fn=<ViewBackward0>)

### To take the result of the prediction and transform it into a traditional numpy array you have to:
- call .detach() to disconnect the tensor from the automatic gradient tracking
- then .numpy()

### The result is an array of 1 cell

In [11]:
prediction_numpy = prediction.detach().numpy()
print("Prediction is {:.2f}".format(prediction_numpy))

Prediction is 0.34


# Train a MF MSE model with PyTorch

# Step 1 Create a Model python object

### The model should implement the forward function which computes the prediction as we did before

In [12]:
class MF_MSE_PyTorch_model(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors):
        super(MF_MSE_PyTorch_model, self).__init__()

        self.n_users = n_users
        self.n_items = n_items

        self.user_factors = torch.nn.Embedding(num_embeddings=self.n_users, embedding_dim=n_factors)
        self.item_factors = torch.nn.Embedding(num_embeddings=self.n_items, embedding_dim=n_factors)

    def forward(self, user_batch, item_batch):
        user_factors_batch = self.user_factors(user_batch)
        item_factors_batch = self.item_factors(item_batch)

        prediction_batch = torch.mul(user_factors_batch, item_factors_batch).sum()

        return prediction_batch

    def get_W(self):
        return self.user_factors.weight.detach().cpu().numpy()

    def get_H(self):
        return self.item_factors.weight.detach().cpu().numpy()

# Step 2 Setup PyTorch devices and Data Reader

In [13]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("MF_MSE_PyTorch: Using CUDA")
else:
    device = torch.device('cpu')
    print("MF_MSE_PyTorch: Using CPU")

MF_MSE_PyTorch: Using CPU


### Create an instance of the model and specify the device it should run on

In [14]:
model = MF_MSE_PyTorch_model(n_users, n_items, num_factors).to(device)

### Choose loss functions (Mean Squared Error in our case), there are quite a few to choose from

In [15]:
lossFunction = torch.nn.MSELoss(reduction="sum")

Alternatively one can implement it 

In [16]:
def _my_MSE_loss(model, user, item):
    
    # Compute prediction for each element in batch
    prediction = model.forward(user, item)

    # Compute total loss for batch
    loss = (prediction - rating).pow(2).mean()
    
    return loss

### Select the optimizer to be used for the model parameters: Adam, AdaGrad, RMSProp etc... 

In [17]:
learning_rate = 1e-4
l2_reg = 1e-3

optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay = l2_reg*learning_rate)

### Define the DatasetInteraction, which will be used to load a specific data point

A DatasetInteraction will implement the Dataset class and provide the \_\_getitem\_\_(self, index) method, which allows to get the data points indexed by that index.

Since we need the data to be a tensor, we pre inizialize everything as a tensor. In practice we save the URM in coordinate format (user, item, rating)

In [18]:
from torch.utils.data import Dataset
import numpy as np

class DatasetInteraction(Dataset):
    def __init__(self, URM):

        URM = URM.tocoo()
        self.n_data_points = URM.nnz

        self._row = torch.tensor(URM.row).type(torch.LongTensor)
        self._col = torch.tensor(URM.col).type(torch.LongTensor)
        self._data = torch.tensor(URM.data).type(torch.FloatTensor)
       
    def __getitem__(self, index):
        return self._row[index], self._col[index], self._data[index]


    def __len__(self):
        return self.n_data_points


### We pass the DatasetIterator to a DataLoader object which manages the use of batches and so on...

In [19]:
from torch.utils.data import DataLoader

# A large batch_size (256, 512...) improves parallelization, but the gradient becomes more smoot
# at some point the performance will increase but at the expense of the final prediction quality
batch_size = 64

dataset_iterator = DatasetInteraction(URM_train)

train_data_loader = DataLoader(dataset=dataset_iterator,
                               batch_size=batch_size,
                               shuffle=True,
                               #num_workers = 2,
                              )

## And now we ran the usual epoch steps
* Data point sampling
* Prediction computation
* Loss function computation
* Gradient computation
* Update

In [20]:
batch = next(iter(train_data_loader))
batch

[tensor([ 2424,  3784,  3405, 10030,  1253, 10183, 12704,   458,  9525,  9593,
          2548,  2416, 10782,  4205, 10095,   623,  1765, 10112,  2277, 11945,
          2902,  2520,  2123,  3839, 10759, 10276,  5187,  2239,  4301,  2054,
          7767,  6565,  3764, 10642,  7016, 10394,  2039,  5933,  2540,  1670,
          8778,  3233, 11981,  1107,  7510,  2953,  9544, 11030,  9637,  7977,
         13020, 10942,  4575, 11890,  2591,  7722,  3956,  8262,  8467, 12476,
          7290,  8467,  5193, 12626]),
 tensor([ 7316,  1128,  2988, 12441,   474, 13893,  4699,  2129, 15240, 18861,
           315,    12,     7,     3,   760,  7266,   261, 14744,   220,  3696,
          1091,   612,    62,  1914,  4491,    88,   141,  4439,   159,  3759,
          1501,  1297,  6200,    59,   539, 16368,    50,  1567,  2387,  2046,
          4482,  1417,    31,   112,  7957,    38,  2637, 21431,  3902,  1071,
          1813, 18114,   849,   269,    59,  2600,   660,   272,  1947, 18501,
          344

In [21]:
%%time
from tqdm.notebook import tqdm

epoch_loss = 0
for batch in tqdm(train_data_loader):

    # Clear previously computed gradients
    optimizer.zero_grad()
    
    user, item, rating = batch
    
    # Compute prediction for each element in batch
    prediction = model.forward(user, item)
    
    # Compute total loss for batch
    loss = (prediction - rating).pow(2).mean()

    # Compute gradients given current loss
    loss.backward()

    # Apply gradient using the selected optimizer
    optimizer.step()

    epoch_loss += loss.item()

  0%|          | 0/5985 [00:00<?, ?it/s]

CPU times: user 16.6 s, sys: 3.96 s, total: 20.5 s
Wall time: 13.5 s


## After the train is complete (it may take a while and many epochs), we can get the matrices in the usual numpy format

In [22]:
user_factors = model.get_W()
item_factors = model.get_H()

In [23]:
user_factors, user_factors.shape

(array([[ 0.42240247, -0.66577035, -0.6934146 , ..., -0.15180531,
          1.2603949 ,  1.1509645 ],
        [ 2.6244314 ,  0.33784166,  0.17933176, ..., -2.460579  ,
          0.3136289 ,  0.50048804],
        [ 0.43939874, -0.33758172,  2.323395  , ..., -0.11025701,
         -2.0287244 ,  0.35721967],
        ...,
        [-0.62914175, -0.64939994, -0.38177714, ..., -0.7151549 ,
         -0.9376109 ,  2.1543765 ],
        [ 0.77411884,  1.6533009 ,  1.0868942 , ...,  1.1801665 ,
          0.6116132 ,  1.1199809 ],
        [ 0.16204336, -1.0171624 , -1.1031039 , ...,  0.40586352,
          0.1465633 , -1.070071  ]], dtype=float32),
 (13025, 10))

In [24]:
item_factors, item_factors.shape

(array([[ 2.1630244 , -0.03907879, -0.12325583, ..., -1.2066027 ,
          0.7551582 ,  0.29103455],
        [-1.0146065 ,  0.9341817 , -1.3799927 , ...,  1.2673644 ,
         -1.5923662 , -0.46877384],
        [-0.3062591 ,  0.4049404 , -0.5016469 , ...,  0.7301209 ,
         -0.4796305 ,  0.94519335],
        ...,
        [ 1.3541249 ,  0.91673934, -0.00678596, ..., -1.3929248 ,
          0.7228819 ,  0.654131  ],
        [-1.487344  , -0.60572374,  1.8419907 , ...,  0.83821696,
         -0.9122638 , -0.2620197 ],
        [-0.6899875 ,  0.620203  ,  0.7397331 , ..., -0.373737  ,
         -0.02342618, -0.7129373 ]], dtype=float32),
 (22348, 10))

### What if I want to change the sampling?
The DatasetInteraction can be modified to obtain the desired behaviour, for example adding some negative (zero-rated) items in the sampling. If we want our model to be able to distinguish between positive and negative items we need to let the model see negative data as well, in our case the negative data is the zero-rated items.

In [25]:
class DatasetInteraction(Dataset):
    def __init__(self, URM_train, positive_quota):
        
        self._URM_train = sps.csr_matrix(URM_train)
        
        URM_train = URM_train.tocoo()
        self.n_data_points = URM_train.nnz

        self._row = torch.tensor(URM_train.row).type(torch.LongTensor)
        self._col = torch.tensor(URM_train.col).type(torch.LongTensor)
        self._data = torch.tensor(URM_train.data).type(torch.FloatTensor)
        self._positive_quota = positive_quota
       
    def __getitem__(self, index):
        select_positive_flag = torch.rand(1, requires_grad=False) > self._positive_quota

        if select_positive_flag[0]:
            return self._row[index], self._col[index], self._data[index]
        else:
            user_id = self._row[index]
            seen_items = self._URM_train.indices[self._URM_train.indptr[user_id]:self._URM_train.indptr[user_id+1]]
            negative_selected = False

            while not negative_selected:
                negative_candidate = torch.randint(low=0, high=self.n_items, size=(1,))[0]

                if negative_candidate not in seen_items:
                    item_negative = negative_candidate
                    negative_selected = True

            return self._row[index], item_negative, torch.tensor(0.0)

        
        return self._row[index], self._col[index], self._data[index]


    def __len__(self):
        return self.n_data_points

You may also change the dataset iterator to one that samples the user profile rather than the specific interaction

In [26]:
class UserProfile_Dataset(Dataset):
    def __init__(self, URM_train, device):
        super().__init__()
        URM_train = sps.csr_matrix(URM_train)
        self.device = device

        self.n_users, self.n_items = URM_train.shape
        self._indptr = URM_train.indptr
        self._indices = torch.tensor(URM_train.indices, dtype = torch.long, device=device)
        self._data = torch.tensor(URM_train.data, dtype = torch.float, device=device)

    def __len__(self):
        return self.n_users

    def __getitem__(self, user_id):
        start_pos = self._indptr[user_id]
        end_pos = self._indptr[user_id+1]

        user_profile = torch.zeros(self.n_items, dtype=torch.float, requires_grad=False, device=self.device)
        user_profile[self._indices[start_pos:end_pos]] = self._data[start_pos:end_pos]

        return user_profile

### What if I want to implement AsySVD? SLIM EN ... 
You just have to change the pytorch model with the desired one (easy to do)

Note these two models work by sampling the whole user profile, they can be adapted to a sampler that provides single interactions

In [27]:
from torch import nn

class AsySVDModel(nn.Module):

    def __init__(self, embedding_size = None, n_items = None, device = None):
        super().__init__()

        self._embedding_item_1 = torch.nn.Parameter(torch.randn((n_items, embedding_size)))
        self._embedding_item_2 = torch.nn.Parameter(torch.randn((embedding_size, n_items)))

    def forward(self, user_profile_batch):
        # input shape is batch_size x n items
        # r_hat_bi = SUM{e=0}{e=embedding_size} SUM{j=0}{j=n items} r_bj * V1_je * V2_ei
        layer_output = torch.einsum("bj,je,ei->bi", user_profile_batch, self._embedding_item_1, self._embedding_item_2)
        return layer_output

In [28]:
class SDenseModel(nn.Module):

    def __init__(self, n_items = None, device = None):
        super().__init__()

        self._S = torch.nn.Parameter(torch.zeros((n_items, n_items)))

    def forward(self, user_profile_batch):
        # input shape is batch_size x n items
        # r_hat_bi = SUM{j=0}{j=n items} r_bj * S_ji
        layer_output = torch.einsum("bj,ji->bi", user_profile_batch, self._S)
        return layer_output

### What if I want to change the loss function?
You can just implement the new one, BPR is quite simple. Make sure that the dataset iterator samples the right data

In [29]:
class BPR_Dataset(Dataset):
    def __init__(self, URM_train):
        super().__init__()
        self._URM_train = sps.csr_matrix(URM_train)
        self.n_users, self.n_items = self._URM_train.shape

    def __len__(self):
        return self.n_users

    def __getitem__(self, user_id):

        seen_items = self._URM_train.indices[self._URM_train.indptr[user_id]:self._URM_train.indptr[user_id+1]]
        item_positive = np.random.choice(seen_items)

        negative_selected = False

        while not negative_selected:
            negative_candidate = np.random.randint(low=0, high=self.n_items, size=1)[0]

            if negative_candidate not in seen_items:
                item_negative = negative_candidate
                negative_selected = True

        return user_id, item_positive, item_negative


In [30]:
def loss_BPR(model, batch):
    user, item_positive, item_negative = batch

    # Compute prediction for each element in batch
    x_ij = model.forward(user, item_positive) - model.forward(user, item_negative)

    # Compute total loss for batch
    loss = -x_ij.sigmoid().log().mean()

    return loss